Q3:

In [38]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os

# Config
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", DEVICE)
BATCH_SIZE = 128
EPOCHS_MLP = 10
EPOCHS_CNN = 10
RSEED = 42
np.random.seed(RSEED)
torch.manual_seed(RSEED)

# Data transforms
transform_mnist = transforms.Compose([
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize((0.1307,), (0.3081,))
])

transform_fashion = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])



In [39]:

# Datasets (full MNIST; you can switch to a subset if compute-limited)
train_mnist = datasets.MNIST(root="./data", train=True, download=True, transform=transform_mnist)
test_mnist = datasets.MNIST(root="./data", train=False, download=True, transform=transform_mnist)

train_fashion = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform_fashion)
test_fashion = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform_fashion)

# Optional: stratified subset helper
def stratified_subset(dataset, n_per_class=None):
    if n_per_class is None:
        return dataset
    targets = np.array([label for _, label in dataset])
    idx_per_class = [np.where(targets == i)[0] for i in range(10)]
    chosen = []
    rng = np.random.default_rng(RSEED)
    for i in range(10):
        idxs = idx_per_class[i]
        k = min(n_per_class, len(idxs))
        chosen.extend(rng.choice(idxs, size=k, replace=False))
    chosen = np.array(sorted(chosen))
    return Subset(dataset, chosen)

# Create loaders (full datasets; replace with stratified_subset(dataset, n_per_class) if needed)
train_loader_mnist = DataLoader(train_mnist, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_mnist  = DataLoader(test_mnist, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

train_loader_fashion = DataLoader(train_fashion, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_fashion  = DataLoader(test_fashion, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [40]:

# 3.1 MLP on MNIST
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        hidden20 = self.fc2(x)  # pre-activation; we'll apply activation next
        x = self.relu(hidden20)
        logits = self.fc3(x)
        return logits, hidden20  # return both logits and 20-dim features

def train_mlp(model, loader, optimizer, criterion, epochs=10):
    model.train()
    for ep in range(epochs):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            logits, _ = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

def eval_mlp(model, loader):
    model.eval()
    preds = []
    truth = []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits, _ = model(xb)
            pred = logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
            truth.extend(yb.cpu().numpy())
    acc = accuracy_score(truth, preds)
    f1 = f1_score(truth, preds, average='macro')
    cm = confusion_matrix(truth, preds)
    return acc, f1, cm, preds, truth

mlp = MLP().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)

# Optional: train on full MNIST or a subset
train_mlp(mlp, train_loader_mnist, optimizer, criterion, epochs=EPOCHS_MLP)

# Extract 20-dim embeddings for t-SNE
def get_mlp_embeddings(model, loader):
    model.eval()
    feats = []
    labels = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            _, h20 = model(xb)
            feats.append(h20.cpu().numpy())
            labels.append(yb.numpy())
    return np.vstack(feats), np.concatenate(labels)

train_emb, train_labels = get_mlp_embeddings(mlp, train_loader_mnist)
test_emb, test_labels = get_mlp_embeddings(mlp, test_loader_mnist)


In [41]:

# t-SNE visualization for trained model
def tsne_visualize(embeddings, labels, title, save_path):
    tsne = TSNE(n_components=2, random_state=RSEED, perplexity=30, max_iter=1000)
    X = tsne.fit_transform(embeddings)
    plt.figure(figsize=(6,6))
    scatter = plt.scatter(X[:,0], X[:,1], c=labels, cmap='tab10', s=5, alpha=0.8)
    plt.legend(*scatter.legend_elements(), title="Digits")
    plt.title(title)
    plt.xlabel("TSNE-1")
    plt.ylabel("TSNE-2")
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

tsne_visualize(train_emb, train_labels, "t-SNE MNIST embeddings (20-dim MLP, trained)", "tsne_mlp_trained.png")

# Untrained model embeddings (reinitialize or use a copy before training)
mlp_untrained = MLP().to(DEVICE)
# Do not train; just get embeddings from random weights
emb_untrained, labels_untrained = get_mlp_embeddings(mlp_untrained, train_loader_mnist)
tsne_visualize(emb_untrained, labels_untrained, "t-SNE MNIST embeddings (20-dim MLP, untrained)", "tsne_mlp_untrained.png")


In [42]:

# Cross-domain: test trained MLP on Fashion-MNIST
# For Fashion-MNIST, we need to adapt input shape/normalization; reuse the same MLP by downsampling Fashion images
def prepare_fashion_loader_for_mlp(loader_fashion):
    # Create a wrapper DataLoader that returns (28x28 grayscale) tensors compatible with MLP input
    return loader_fashion

fashion_loader_for_mlp = prepare_fashion_loader_for_mlp(test_loader_fashion)  # test on Fashion-MNIST
# Evaluate: we need to ensure label space matches (10 classes identical)
acc_fashion, f1_fashion, cm_fashion, preds_f, truth_f = eval_mlp(mlp, fashion_loader_for_mlp)
print(f"MNIST MLP on Fashion-MNIST - Acc: {acc_fashion:.4f}, F1: {f1_fashion:.4f}")
# Note: t-SNE on Fashion embeddings
emb_fashion, label_fashion = get_mlp_embeddings(mlp, fashion_loader_for_mlp)
tsne_visualize(emb_fashion, label_fashion, "t-SNE MNIST-trained MLP on Fashion-MNIST embeddings", "tsne_mnist_mlp_on_fashion.png")


MNIST MLP on Fashion-MNIST - Acc: 0.0791, F1: 0.0529


In [43]:
# Confusion matrices saved as images
def save_cm(cm, fname):
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.title("Confusion Matrix")
    plt.colorbar()
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()

In [44]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Prepare flattened data from MNIST loaders for baseline models
def prepare_flat_data(loader):
    X, y = [], []
    for xb, yb in loader:
        xb_cpu = xb.view(xb.size(0), -1).cpu().numpy()  # flatten and convert to numpy
        y_cpu = yb.cpu().numpy()
        X.append(xb_cpu)
        y.append(y_cpu)
    return np.vstack(X), np.concatenate(y)

# Prepare train and test data
X_train_mlp, y_train_mlp = prepare_flat_data(train_loader_mnist)
X_test_mlp, y_test_mlp = prepare_flat_data(test_loader_mnist)

# Train Random Forest on flattened MNIST data
rf_clf = RandomForestClassifier(random_state=RSEED, n_jobs=-1)
rf_clf.fit(X_train_mlp, y_train_mlp)

# Evaluate Random Forest
rf_preds = rf_clf.predict(X_test_mlp)
rf_acc = accuracy_score(y_test_mlp, rf_preds)
rf_f1 = f1_score(y_test_mlp, rf_preds, average='macro')
rf_cm = confusion_matrix(y_test_mlp, rf_preds)
print(f"Random Forest - Accuracy: {rf_acc:.4f}, F1-score: {rf_f1:.4f}")

# Train Logistic Regression on flattened MNIST data
lr_clf = LogisticRegression(random_state=RSEED, max_iter=200, n_jobs=-1, solver='lbfgs', multi_class='multinomial')
lr_clf.fit(X_train_mlp, y_train_mlp)

# Evaluate Logistic Regression
lr_preds = lr_clf.predict(X_test_mlp)
lr_acc = accuracy_score(y_test_mlp, lr_preds)
lr_f1 = f1_score(y_test_mlp, lr_preds, average='macro')
lr_cm = confusion_matrix(y_test_mlp, lr_preds)
print(f"Logistic Regression - Accuracy: {lr_acc:.4f}, F1-score: {lr_f1:.4f}")

# Save confusion matrices using your existing save_cm function
save_cm(rf_cm, "outputs/cm_rf.png")
save_cm(lr_cm, "outputs/cm_lr.png")


Random Forest - Accuracy: 0.9684, F1-score: 0.9682


c:\Users\karti\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression - Accuracy: 0.9239, F1-score: 0.9228


In [27]:

# 3.2 CNN on MNIST
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 14x14
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

def train_cnn(model, loader, optimizer, criterion, epochs=10):
    model.train()
    for ep in range(epochs):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

def eval_cnn(model, loader):
    model.eval()
    preds = []
    truth = []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            pred = logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
            truth.extend(yb.cpu().numpy())
    acc = accuracy_score(truth, preds)
    f1 = f1_score(truth, preds, average='macro')
    cm = confusion_matrix(truth, preds)
    return acc, f1, cm, preds, truth

cnn = SimpleCNN().to(DEVICE)
optimizer_cnn = optim.Adam(cnn.parameters(), lr=0.001)
criterion_cnn = nn.CrossEntropyLoss()
train_loader_mnist_for_cnn = DataLoader(train_mnist, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_mnist_for_cnn = DataLoader(test_mnist, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

train_cnn(cnn, train_loader_mnist_for_cnn, optimizer_cnn, criterion_cnn, epochs=EPOCHS_CNN)
acc_mnist_cnn, f1_mnist_cnn, cm_mnist_cnn, preds_cnn, truth_cnn = eval_cnn(cnn, test_loader_mnist_for_cnn)
print(f"CNN MNIST - Acc: {acc_mnist_cnn:.4f}, F1: {f1_mnist_cnn:.4f}")


CNN MNIST - Acc: 0.9850, F1: 0.9849


In [28]:

# Pretrained CNNs for inference
# Depending on environment, you may load lightweight pretrained models and adapt 28x28 grayscale to 3-channel if needed.
# Example: use a pretrained SmallVGG-like or MobileNet; here is a placeholder approach:
# - If using PyTorch, you can upsample grayscale to 3 channels and resize to 224x224, then run through pretrained model.
# For portability, you can skip actual pretrained inference if not available.
# Below is a safe scaffold that you can fill in with actual pretrained model loading.

def infer_pretrained(imgs, model):
    # imgs: tensor [N,1,28,28], convert to required input for model
    with torch.no_grad():
        model.eval()
        # Implement according to model input requirements
        # Return logits or probabilities
        pass

# Example of parameter counts
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

mlp_params = count_parameters(mlp)
cnn_params = count_parameters(cnn)
print(f"Model sizes - MLP params: {mlp_params}, CNN params: {cnn_params}")

# Inference timing on MNIST test set for all three models
def measure_inference_time(model, loader):
    model.eval()
    t0 = time.time()
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            _ = model(xb)
    t1 = time.time()
    return (t1 - t0)

# Time for MLP inference on MNIST test
time_mlp_infer = measure_inference_time(mlp, test_loader_mnist)
print(f"MLP inference time on MNIST test: {time_mlp_infer:.4f} seconds")

# Time for CNN inference on MNIST test
def cnn_forward_time(model, loader):
    model.eval()
    t0 = time.time()
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            _ = model(xb)
    t1 = time.time()
    return t1 - t0

time_cnn_infer = cnn_forward_time(cnn, test_loader_mnist_for_cnn)
print(f"CNN inference time on MNIST test: {time_cnn_infer:.4f} seconds")


Model sizes - MLP params: 24380, CNN params: 804554
MLP inference time on MNIST test: 1.0478 seconds
CNN inference time on MNIST test: 4.7820 seconds


In [29]:

# Confidence, results storage
os.makedirs("outputs", exist_ok=True)
np.save("outputs/mlp_embeddings_train.npy", train_emb)
np.save("outputs/mlp_embeddings_test.npy", test_emb)
np.save("outputs/mlp_train_labels.npy", train_labels)
np.save("outputs/mlp_test_labels.npy", test_labels)
np.save("outputs/fashion_embeddings.npy", emb_fashion)

# Confusion matrices saved as images
def save_cm(cm, fname):
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.title("Confusion Matrix")
    plt.colorbar()
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()

save_cm(cm_mnist_cnn, "outputs/cm_mnist_cnn.png")
save_cm(cm_mnist_cnn, "outputs/cm_mlp_and_cnn.png")  # reuse for consistency
save_cm(cm_fashion, "outputs/cm_fashion.png")  # if you compute it later

print("All done. Results saved to outputs/ directory.")


All done. Results saved to outputs/ directory.


In [54]:
# from torchvision import models
# from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# # Use safe test loader with num_workers=0 to avoid multiprocessing issues
# test_loader_mnist_safe = DataLoader(test_mnist, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# # Load pretrained MobileNetV2 and adjust final layer for 10 classes
# mobilenet = models.mobilenet_v2(pretrained=True)
# mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, 10)
# mobilenet = mobilenet.to(DEVICE).eval()

# # MobileNetV2 inference
# preds_mobilenet = []
# labels_mobilenet = []

# with torch.no_grad():
#     for xb, yb in test_loader_mnist_safe:
#         xb = nn.functional.interpolate(xb, size=(224, 224), mode='bilinear')
#         if xb.shape[1] == 1:
#             xb = xb.repeat(1, 3, 1, 1)  # convert grayscale to RGB
#         xb = xb.to(DEVICE)
#         logits = mobilenet(xb)
#         preds = torch.argmax(logits, dim=1).cpu().numpy()
#         preds_mobilenet.extend(preds)
#         labels_mobilenet.extend(yb.numpy())

# mobilenet_acc = accuracy_score(labels_mobilenet, preds_mobilenet)
# mobilenet_f1 = f1_score(labels_mobilenet, preds_mobilenet, average='macro')
# mobilenet_cm = confusion_matrix(labels_mobilenet, preds_mobilenet)
# print(f"MobileNetV2 - Acc: {mobilenet_acc:.4f}, F1: {mobilenet_f1:.4f}")

# # Load pretrained EfficientNet and adjust final layer for 10 classes
# efficientnet = models.efficientnet_b0(pretrained=True)
# efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 10)
# efficientnet = efficientnet.to(DEVICE).eval()

# # EfficientNet inference
# preds_effnet = []
# labels_effnet = []

# with torch.no_grad():
#     for xb, yb in test_loader_mnist_safe:
#         xb = nn.functional.interpolate(xb, size=(224, 224), mode='bilinear')
#         if xb.shape[1] == 1:
#             xb = xb.repeat(1, 3, 1, 1)
#         xb = xb.to(DEVICE)
#         logits = efficientnet(xb)
#         preds = torch.argmax(logits, dim=1).cpu().numpy()
#         preds_effnet.extend(preds)
#         labels_effnet.extend(yb.numpy())

# effnet_acc = accuracy_score(labels_effnet, preds_effnet)
# effnet_f1 = f1_score(labels_effnet, preds_effnet, average='macro')
# effnet_cm = confusion_matrix(labels_effnet, preds_effnet)
# print(f"EfficientNet - Acc: {effnet_acc:.4f}, F1: {effnet_f1:.4f}")

# # Save confusion matrices
# save_cm(mobilenet_cm, "outputs/cm_mobilenet.png")
# save_cm(effnet_cm, "outputs/cm_efficientnet.png")

# # Model size (parameter count)
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters())

# print(f"MobileNetV2 params: {count_parameters(mobilenet)}")
# print(f"EfficientNet params: {count_parameters(efficientnet)}")

# # Inference time measurement
# import time

# def measure_inference_time(model, loader):
#     model.eval()
#     t0 = time.time()
#     with torch.no_grad():
#         for xb, _ in loader:
#             xb = nn.functional.interpolate(xb, size=(224, 224), mode='bilinear')
#             if xb.shape[1] == 1:
#                 xb = xb.repeat(1, 3, 1, 1)
#             xb = xb.to(DEVICE)
#             _ = model(xb)
#     t1 = time.time()
#     return t1 - t0

# mobilenet_time = measure_inference_time(mobilenet, test_loader_mnist_safe)
# effnet_time = measure_inference_time(efficientnet, test_loader_mnist_safe)
# print(f"MobileNetV2 inference time: {mobilenet_time:.4f} seconds")
# print(f"EfficientNet inference time: {effnet_time:.4f} seconds")


In [53]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import time

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
NUM_EPOCHS = 5
LR = 0.001

# Data transforms to match ImageNet-pretrained models
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Converts MNIST to 3 channel
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet stats
                         [0.229, 0.224, 0.225]),
])

# Load MNIST datasets
train_mnist = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_mnist = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_mnist, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_mnist, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# MobileNetV2 setup
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, 10)
mobilenet = mobilenet.to(DEVICE)

# EfficientNet setup
efficientnet = models.efficientnet_b0(pretrained=True)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 10)
efficientnet = efficientnet.to(DEVICE)

# Freeze backbone parameters if you only want to train classifier:
for param in mobilenet.features.parameters():
    param.requires_grad = False
for param in efficientnet.features.parameters():
    param.requires_grad = False

# Setup optimizer and loss
optimizer_mobilenet = torch.optim.Adam(mobilenet.classifier.parameters(), lr=LR)
optimizer_efficientnet = torch.optim.Adam(efficientnet.classifier.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Training loop function
def train_model(model, optimizer, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

# Train both models
train_model(mobilenet, optimizer_mobilenet, train_loader, NUM_EPOCHS)
train_model(efficientnet, optimizer_efficientnet, train_loader, NUM_EPOCHS)

# Evaluation/inference
def evaluate_model(model, test_loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            preds.extend(pred)
            labels.extend(yb.numpy())
    return preds, labels

preds_mobilenet, labels_mobilenet = evaluate_model(mobilenet, test_loader)
mobilenet_acc = accuracy_score(labels_mobilenet, preds_mobilenet)
mobilenet_f1 = f1_score(labels_mobilenet, preds_mobilenet, average='macro')
mobilenet_cm = confusion_matrix(labels_mobilenet, preds_mobilenet)
print(f"MobileNetV2 - Acc: {mobilenet_acc:.4f}, F1: {mobilenet_f1:.4f}")

preds_effnet, labels_effnet = evaluate_model(efficientnet, test_loader)
effnet_acc = accuracy_score(labels_effnet, preds_effnet)
effnet_f1 = f1_score(labels_effnet, preds_effnet, average='macro')
effnet_cm = confusion_matrix(labels_effnet, preds_effnet)
print(f"EfficientNet - Acc: {effnet_acc:.4f}, F1: {effnet_f1:.4f}")

# Model size (parameter count)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())
print(f"MobileNetV2 params: {count_parameters(mobilenet)}")
print(f"EfficientNet params: {count_parameters(efficientnet)}")

# Inference time measurement
def measure_inference_time(model, loader):
    model.eval()
    t0 = time.time()
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            _ = model(xb)
    t1 = time.time()
    return t1 - t0
mobilenet_time = measure_inference_time(mobilenet, test_loader)
effnet_time = measure_inference_time(efficientnet, test_loader)
print(f"MobileNetV2 inference time: {mobilenet_time:.4f} seconds")
print(f"EfficientNet inference time: {effnet_time:.4f} seconds")


c:\Users\karti\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\karti\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\karti\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the m

MobileNetV2 - Acc: 0.9655, F1: 0.9652
EfficientNet - Acc: 0.9631, F1: 0.9628
MobileNetV2 params: 2236682
EfficientNet params: 4020358
MobileNetV2 inference time: 11.6316 seconds
EfficientNet inference time: 11.8338 seconds
